# Search Analytics with DuckDB

This notebook helps you analyze your Search Analytics data using DuckDB.

---
# QUICK START

**Just run these 2 cells to get started!**

### Step 1: Set the path to your CSV file

In [ ]:
#################################################################
#  ONLY SETTING: Path to your CSV file                         #
#################################################################

CSV_PATH = '../data/search_export.csv'    # <-- CHANGE THIS!

#################################################################

### Step 2: Run this cell - everything else happens automatically

In [ ]:
# ===== AUTOMATIC SETUP =====
# This cell:
# 1. Imports all required libraries
# 2. Creates the database (.db file)
# 3. Reads your CSV and automatically creates a table
# 4. Normalizes column names (user_Id -> user_id, etc.)
# 5. Converts German date formats (DD.MM.YYYY HH:MM) automatically
# 6. Creates a session key for easy grouping
# 7. Shows you what was imported

import duckdb
import pandas as pd
import re
from pathlib import Path

# Plotting optional
try:
    import matplotlib.pyplot as plt
    import matplotlib.dates as mdates
    plt.style.use('seaborn-v0_8-whitegrid')
    PLOTTING_AVAILABLE = True
except ImportError:
    PLOTTING_AVAILABLE = False

# Pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Create database (in same folder as CSV)
csv_path = Path(CSV_PATH)
db_path = csv_path.parent / 'searchanalytics.db'

# Establish connection
con = duckdb.connect(str(db_path))

# Helper functions
def query(sql):
    """Execute SQL and return DataFrame"""
    return con.execute(sql).df()

def execute(sql):
    """Execute SQL without return"""
    con.execute(sql)

# Create table from CSV
print("="*60)
print("AUTOMATIC IMPORT")
print("="*60)

# Drop table if exists, then recreate
execute("DROP TABLE IF EXISTS searches")

# Read CSV
execute(f"""
    CREATE TABLE searches AS
    SELECT * FROM read_csv('{CSV_PATH}', auto_detect=true)
""")

# Normalize column names (user_Id -> user_id, session_Id -> session_id)
print("\nNormalizing column names...")
schema = query("DESCRIBE searches")
col_names = schema['column_name'].tolist()

rename_map = {
    'user_Id': 'user_id',
    'session_Id': 'session_id'
}

renamed_cols = []
for old_name, new_name in rename_map.items():
    if old_name in col_names:
        execute(f"ALTER TABLE searches RENAME COLUMN {old_name} TO {new_name}")
        renamed_cols.append(f"{old_name} → {new_name}")

if renamed_cols:
    print(f"  Renamed: {', '.join(renamed_cols)}")
else:
    print("  No renaming needed")

# Automatically convert German date formats (DD.MM.YYYY or DD.MM.YYYY HH:MM)
print("\nChecking for German date formats...")
schema = query("DESCRIBE searches")
varchar_cols = schema[schema['column_type'] == 'VARCHAR']['column_name'].tolist()

converted_cols = []
for col in varchar_cols:
    # Check first non-null value
    sample = query(f"SELECT {col} FROM searches WHERE {col} IS NOT NULL LIMIT 1")
    if len(sample) > 0:
        val = str(sample.iloc[0, 0])
        # Detect German date: DD.MM.YYYY or DD.MM.YYYY HH:MM
        if re.match(r'^\d{2}\.\d{2}\.\d{4}', val):
            try:
                # Determine format
                if re.match(r'^\d{2}\.\d{2}\.\d{4} \d{2}:\d{2}(:\d{2})?$', val):
                    # With time
                    if val.count(':') == 2:
                        fmt = '%d.%m.%Y %H:%M:%S'
                    else:
                        fmt = '%d.%m.%Y %H:%M'
                else:
                    # Date only
                    fmt = '%d.%m.%Y'
                
                # Convert
                execute(f"ALTER TABLE searches ADD COLUMN {col}_temp TIMESTAMP")
                execute(f"UPDATE searches SET {col}_temp = strptime({col}, '{fmt}')")
                execute(f"ALTER TABLE searches DROP COLUMN {col}")
                execute(f"ALTER TABLE searches RENAME COLUMN {col}_temp TO {col}")
                converted_cols.append(col)
            except Exception as e:
                print(f"  Warning: Could not convert {col}: {e}")

if converted_cols:
    print(f"  Converted: {', '.join(converted_cols)}")
else:
    print("  No German date formats found")

# Create session key (if user_id and session_id exist)
print("\nChecking for session fields...")
schema = query("DESCRIBE searches")
col_names = schema['column_name'].tolist()

has_user_id = 'user_id' in col_names
has_session_id = 'session_id' in col_names
has_timestamp = 'timestamp' in col_names

if has_user_id and has_session_id and has_timestamp:
    execute("""
        ALTER TABLE searches ADD COLUMN session_date DATE;
    """)
    execute("""
        UPDATE searches SET session_date = DATE_TRUNC('day', timestamp)::DATE;
    """)
    execute("""
        ALTER TABLE searches ADD COLUMN session_key VARCHAR;
    """)
    execute("""
        UPDATE searches SET session_key = 
            COALESCE(CAST(session_date AS VARCHAR), '') || '_' || 
            COALESCE(user_id, '') || '_' || 
            COALESCE(session_id, '');
    """)
    print("  session_date created (day from timestamp)")
    print("  session_key created (session_date + user_id + session_id)")
    print("  → Group with: GROUP BY session_key")
elif has_user_id and has_session_id:
    execute("""
        ALTER TABLE searches ADD COLUMN session_key VARCHAR;
    """)
    execute("""
        UPDATE searches SET session_key = 
            COALESCE(user_id, '') || '_' || 
            COALESCE(session_id, '');
    """)
    print("  session_key created (user_id + session_id)")
    print("  Note: No timestamp found, session_date not created")
else:
    missing = []
    if not has_user_id: missing.append('user_id')
    if not has_session_id: missing.append('session_id')
    print(f"  Missing columns: {', '.join(missing)}")
    print("  → session_key not created")

# What was imported?
row_count = query("SELECT COUNT(*) as n FROM searches")['n'][0]
print(f"\n CSV file: {csv_path.name}")
print(f" Database: {db_path.name}")
print(f" Imported: {row_count:,} rows")

print("\n" + "="*60)
print("DETECTED COLUMNS")
print("="*60)
schema = query("DESCRIBE searches")
for _, row in schema.iterrows():
    print(f"  {row['column_name']:30} {row['column_type']}")

print("\n" + "="*60)
print("FIRST 5 ROWS")
print("="*60)
display(query("SELECT * FROM searches LIMIT 5"))

print("\n Setup complete! You can now run the analysis cells below.")

---
# ANALYSES

From here you can run the cells that interest you.

**Important:** If your columns have different names, adjust them in the queries!
Typical column variants:
- Timestamp: `timestamp`, `date`, `datetime`, `created_at`
- Search term: `search_query`, `query`, `search_term`, `keyword`
- Results: `results_count`, `result_count`, `hits`, `total_results`
- Response time: `response_time`, `duration`, `latency_ms`

---
## Basic Statistics

In [ ]:
# Overview: What do we have?
query("DESCRIBE searches")

In [ ]:
# First and last entries
query("""
    SELECT
        COUNT(*) as total_rows,
        MIN(timestamp) as first_entry,
        MAX(timestamp) as last_entry
    FROM searches
""")

In [ ]:
# View sample data
query("SELECT * FROM searches LIMIT 20")

---
## Time Distribution

In [ ]:
# Entries per day
# NOTE: Replace 'timestamp' with your date column if needed

query("""
    SELECT
        DATE_TRUNC('day', timestamp)::DATE as date,
        COUNT(*) as count
    FROM searches
    GROUP BY 1
    ORDER BY 1 DESC
    LIMIT 30
""")

In [ ]:
# Distribution by hour
query("""
    SELECT
        EXTRACT(HOUR FROM timestamp) as hour,
        COUNT(*) as count,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 2) as percent
    FROM searches
    GROUP BY 1
    ORDER BY 1
""")

In [ ]:
# Distribution by weekday
query("""
    SELECT
        DAYNAME(timestamp) as weekday,
        DAYOFWEEK(timestamp) as day_nr,
        COUNT(*) as count
    FROM searches
    GROUP BY 1, 2
    ORDER BY 2
""")

---
## Session Analyses

Analyses based on user sessions (grouped by session_key = day + user_id + session_id)

In [ ]:
# Session overview: How many sessions, how many searches per session?
query("""
    SELECT
        COUNT(DISTINCT session_key) as total_sessions,
        COUNT(*) as total_searches,
        ROUND(COUNT(*) * 1.0 / COUNT(DISTINCT session_key), 1) as avg_searches_per_session,
        COUNT(DISTINCT user_id) as unique_users
    FROM searches
""")

In [ ]:
# Distribution: Number of searches per session
query("""
    SELECT
        searches_in_session,
        COUNT(*) as session_count,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 1) as percent
    FROM (
        SELECT session_key, COUNT(*) as searches_in_session
        FROM searches
        GROUP BY session_key
    )
    GROUP BY 1
    ORDER BY 1
    LIMIT 20
""")

In [ ]:
# Sessions with most searches (power users / problems?)
query("""
    SELECT
        session_key,
        session_date,
        user_id,
        session_id,
        COUNT(*) as search_count,
        MIN(timestamp) as first_search,
        MAX(timestamp) as last_search,
        DATEDIFF('minute', MIN(timestamp), MAX(timestamp)) as duration_minutes
    FROM searches
    GROUP BY session_key, session_date, user_id, session_id
    ORDER BY search_count DESC
    LIMIT 20
""")

In [ ]:
# User activity: How many sessions per user?
query("""
    SELECT
        user_id,
        COUNT(DISTINCT session_key) as session_count,
        COUNT(*) as total_searches,
        ROUND(COUNT(*) * 1.0 / COUNT(DISTINCT session_key), 1) as searches_per_session,
        MIN(session_date) as first_session,
        MAX(session_date) as last_session
    FROM searches
    GROUP BY user_id
    ORDER BY session_count DESC
    LIMIT 20
""")

In [ ]:
# Session timeline: Show all activities for a specific session
# Replace session_key with a value from the query above

SESSION_KEY = 'ENTER_SESSION_KEY_HERE'  # <-- Change this

query(f"""
    SELECT *
    FROM searches
    WHERE session_key = '{SESSION_KEY}'
    ORDER BY timestamp
""")

In [ ]:
# Sessions per day
query("""
    SELECT
        session_date as date,
        COUNT(DISTINCT session_key) as sessions,
        COUNT(DISTINCT user_id) as unique_users,
        COUNT(*) as total_searches,
        ROUND(COUNT(*) * 1.0 / COUNT(DISTINCT session_key), 1) as searches_per_session
    FROM searches
    GROUP BY 1
    ORDER BY 1 DESC
    LIMIT 30
""")

---
## Search Journey Analysis

Analyzes the complete search flow per session: Search → Results → Click?

**Event types:**
- `SEARCH_TRIGGERED` / `SEARCH_STARTED` - User starts search
- `SEARCH_RESULT_COUNT` - Results are displayed
- `SEARCH_TAB_CLICK`, `SEARCH_ALL_TAB_PAGE_CLICK`, `SEARCH_NEWS_TAB_PAGE_CLICK`, `SEARCH_GOTO_TAB_PAGE_CLICK` - User clicks on result

**Success = Search leads to click**

In [ ]:
# Overview: What event types do we have?
query("""
    SELECT 
        name as event_type,
        COUNT(*) as count,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 1) as percent
    FROM searches
    GROUP BY 1
    ORDER BY 2 DESC
""")

In [ ]:
# Search Funnel: How many searches → results → clicks?
query("""
    SELECT
        COUNT(DISTINCT CASE WHEN name IN ('SEARCH_TRIGGERED', 'SEARCH_STARTED') THEN session_key END) as sessions_with_search,
        COUNT(DISTINCT CASE WHEN name = 'SEARCH_RESULT_COUNT' THEN session_key END) as sessions_with_results,
        COUNT(DISTINCT CASE WHEN name IN ('SEARCH_TAB_CLICK', 'SEARCH_ALL_TAB_PAGE_CLICK', 'SEARCH_NEWS_TAB_PAGE_CLICK', 'SEARCH_GOTO_TAB_PAGE_CLICK') THEN session_key END) as sessions_with_click,
        ROUND(100.0 * COUNT(DISTINCT CASE WHEN name IN ('SEARCH_TAB_CLICK', 'SEARCH_ALL_TAB_PAGE_CLICK', 'SEARCH_NEWS_TAB_PAGE_CLICK', 'SEARCH_GOTO_TAB_PAGE_CLICK') THEN session_key END) 
            / NULLIF(COUNT(DISTINCT CASE WHEN name IN ('SEARCH_TRIGGERED', 'SEARCH_STARTED') THEN session_key END), 0), 1) as click_through_rate_pct
    FROM searches
""")

In [ ]:
# Null-result searches: Which search terms return 0 results?
query("""
    SELECT 
        COALESCE(CP_searchQuery, searchQuery, query) as search_term,
        COUNT(*) as count,
        AVG(CAST(CP_totalResultCount AS INTEGER)) as avg_results,
        SUM(CASE WHEN CAST(CP_totalResultCount AS INTEGER) = 0 THEN 1 ELSE 0 END) as null_result_count,
        ROUND(100.0 * SUM(CASE WHEN CAST(CP_totalResultCount AS INTEGER) = 0 THEN 1 ELSE 0 END) / COUNT(*), 1) as null_rate_pct
    FROM searches
    WHERE name = 'SEARCH_RESULT_COUNT'
    GROUP BY 1
    HAVING COUNT(*) >= 3
    ORDER BY null_result_count DESC
    LIMIT 30
""")

In [ ]:
# Top search terms with success rate (click after search)
query("""
    WITH searches_with_query AS (
        SELECT 
            session_key,
            COALESCE(CP_searchQuery, searchQuery, query) as search_term,
            name,
            timestamp
        FROM searches
        WHERE COALESCE(CP_searchQuery, searchQuery, query) IS NOT NULL
    ),
    search_events AS (
        SELECT DISTINCT session_key, search_term
        FROM searches_with_query 
        WHERE name IN ('SEARCH_TRIGGERED', 'SEARCH_STARTED', 'SEARCH_RESULT_COUNT')
    ),
    click_events AS (
        SELECT DISTINCT session_key
        FROM searches_with_query
        WHERE name IN ('SEARCH_TAB_CLICK', 'SEARCH_ALL_TAB_PAGE_CLICK', 'SEARCH_NEWS_TAB_PAGE_CLICK', 'SEARCH_GOTO_TAB_PAGE_CLICK')
    )
    SELECT 
        s.search_term,
        COUNT(*) as search_count,
        SUM(CASE WHEN c.session_key IS NOT NULL THEN 1 ELSE 0 END) as with_click,
        ROUND(100.0 * SUM(CASE WHEN c.session_key IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*), 1) as success_rate_pct
    FROM search_events s
    LEFT JOIN click_events c ON s.session_key = c.session_key
    GROUP BY 1
    HAVING COUNT(*) >= 5
    ORDER BY search_count DESC
    LIMIT 30
""")

In [ ]:
# Problematic searches: Many results but no clicks (user doesn't find what they need)
query("""
    WITH search_results AS (
        SELECT 
            session_key,
            COALESCE(CP_searchQuery, searchQuery, query) as search_term,
            CAST(CP_totalResultCount AS INTEGER) as total_results
        FROM searches
        WHERE name = 'SEARCH_RESULT_COUNT'
          AND CAST(CP_totalResultCount AS INTEGER) > 0
    ),
    click_events AS (
        SELECT DISTINCT session_key
        FROM searches
        WHERE name IN ('SEARCH_TAB_CLICK', 'SEARCH_ALL_TAB_PAGE_CLICK', 'SEARCH_NEWS_TAB_PAGE_CLICK', 'SEARCH_GOTO_TAB_PAGE_CLICK')
    )
    SELECT 
        sr.search_term,
        COUNT(*) as search_count,
        ROUND(AVG(sr.total_results), 0) as avg_results,
        SUM(CASE WHEN c.session_key IS NULL THEN 1 ELSE 0 END) as without_click,
        ROUND(100.0 * SUM(CASE WHEN c.session_key IS NULL THEN 1 ELSE 0 END) / COUNT(*), 1) as abandon_rate_pct
    FROM search_results sr
    LEFT JOIN click_events c ON sr.session_key = c.session_key
    GROUP BY 1
    HAVING COUNT(*) >= 5 AND SUM(CASE WHEN c.session_key IS NULL THEN 1 ELSE 0 END) > 0
    ORDER BY without_click DESC
    LIMIT 30
""")

In [ ]:
# Search reformulations: Sessions with multiple searches (user needs to adjust query)
query("""
    WITH session_searches AS (
        SELECT 
            session_key,
            COUNT(DISTINCT COALESCE(CP_searchQuery, searchQuery, query)) as unique_queries,
            COUNT(*) as total_search_events
        FROM searches
        WHERE name IN ('SEARCH_TRIGGERED', 'SEARCH_STARTED')
          AND COALESCE(CP_searchQuery, searchQuery, query) IS NOT NULL
        GROUP BY 1
        HAVING COUNT(DISTINCT COALESCE(CP_searchQuery, searchQuery, query)) > 1
    )
    SELECT 
        unique_queries as different_search_count,
        COUNT(*) as sessions,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 1) as percent
    FROM session_searches
    GROUP BY 1
    ORDER BY 1
""")

In [ ]:
# Complete journey of a session
# Choose a session_key from above

SESSION_KEY = 'ENTER_SESSION_KEY_HERE'  # <-- Change this

query(f"""
    SELECT 
        timestamp,
        name as event,
        COALESCE(CP_searchQuery, searchQuery, query) as search_term,
        CP_totalResultCount as results,
        CP_peopleResultCount as people,
        CP_newsResultCount as news,
        CP_gotoResultCount as goto
    FROM searches
    WHERE session_key = '{SESSION_KEY}'
    ORDER BY timestamp
""")

In [ ]:
# Result distribution by category: Where do results come from?
query("""
    SELECT
        'Total' as category,
        COUNT(*) as searches_with_result,
        ROUND(AVG(CAST(CP_totalResultCount AS FLOAT)), 1) as avg_count,
        SUM(CASE WHEN CAST(CP_totalResultCount AS INTEGER) = 0 THEN 1 ELSE 0 END) as null_results
    FROM searches WHERE name = 'SEARCH_RESULT_COUNT'
    
    UNION ALL
    
    SELECT 'People', COUNT(*), ROUND(AVG(CAST(CP_peopleResultCount AS FLOAT)), 1),
        SUM(CASE WHEN CAST(CP_peopleResultCount AS INTEGER) = 0 THEN 1 ELSE 0 END)
    FROM searches WHERE name = 'SEARCH_RESULT_COUNT' AND CP_peopleResultCount IS NOT NULL
    
    UNION ALL
    
    SELECT 'News', COUNT(*), ROUND(AVG(CAST(CP_newsResultCount AS FLOAT)), 1),
        SUM(CASE WHEN CAST(CP_newsResultCount AS INTEGER) = 0 THEN 1 ELSE 0 END)
    FROM searches WHERE name = 'SEARCH_RESULT_COUNT' AND CP_newsResultCount IS NOT NULL
    
    UNION ALL
    
    SELECT 'Intranet News', COUNT(*), ROUND(AVG(CAST(CP_intranetNewsResultCount AS FLOAT)), 1),
        SUM(CASE WHEN CAST(CP_intranetNewsResultCount AS INTEGER) = 0 THEN 1 ELSE 0 END)
    FROM searches WHERE name = 'SEARCH_RESULT_COUNT' AND CP_intranetNewsResultCount IS NOT NULL
    
    UNION ALL
    
    SELECT 'GoTo', COUNT(*), ROUND(AVG(CAST(CP_gotoResultCount AS FLOAT)), 1),
        SUM(CASE WHEN CAST(CP_gotoResultCount AS INTEGER) = 0 THEN 1 ELSE 0 END)
    FROM searches WHERE name = 'SEARCH_RESULT_COUNT' AND CP_gotoResultCount IS NOT NULL
""")

In [ ]:
# Click distribution: Which result categories are clicked?
query("""
    SELECT 
        name as click_type,
        COUNT(*) as count,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 1) as percent
    FROM searches
    WHERE name IN ('SEARCH_TAB_CLICK', 'SEARCH_ALL_TAB_PAGE_CLICK', 'SEARCH_NEWS_TAB_PAGE_CLICK', 'SEARCH_GOTO_TAB_PAGE_CLICK')
    GROUP BY 1
    ORDER BY 2 DESC
""")

---
## Top Values (adjustable for any column)

In [ ]:
# Top 20 most frequent values of a column
# NOTE: Replace 'search_query' with the column you're interested in

COLUMN = 'search_query'  # <-- Change this

query(f"""
    SELECT
        {COLUMN},
        COUNT(*) as count,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 2) as percent
    FROM searches
    WHERE {COLUMN} IS NOT NULL
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
""")

---
## Null-Result Analysis

If you have a column with result count (e.g. `results_count`)

In [ ]:
# Overall null rate
# NOTE: Replace 'results_count' with your column

query("""
    SELECT
        COUNT(*) as total_searches,
        SUM(CASE WHEN results_count = 0 THEN 1 ELSE 0 END) as null_results,
        ROUND(100.0 * SUM(CASE WHEN results_count = 0 THEN 1 ELSE 0 END) / COUNT(*), 2) as null_rate_pct
    FROM searches
""")

In [ ]:
# Search terms with most null results
query("""
    SELECT
        search_query,
        COUNT(*) as count,
        SUM(CASE WHEN results_count = 0 THEN 1 ELSE 0 END) as null_results,
        ROUND(100.0 * SUM(CASE WHEN results_count = 0 THEN 1 ELSE 0 END) / COUNT(*), 1) as null_rate_pct
    FROM searches
    WHERE search_query IS NOT NULL
    GROUP BY 1
    HAVING COUNT(*) >= 5  -- At least 5 searches
    ORDER BY null_results DESC
    LIMIT 20
""")

In [ ]:
# Null rate per day
query("""
    SELECT
        DATE_TRUNC('day', timestamp)::DATE as date,
        COUNT(*) as total,
        SUM(CASE WHEN results_count = 0 THEN 1 ELSE 0 END) as null_results,
        ROUND(100.0 * SUM(CASE WHEN results_count = 0 THEN 1 ELSE 0 END) / COUNT(*), 2) as null_rate_pct
    FROM searches
    GROUP BY 1
    ORDER BY 1 DESC
    LIMIT 30
""")

---
## Export Search Journey Analysis (Excel)

Exports all journey analyses to an Excel file with separate tabs.

In [ ]:
# Search Journey Analysis → Excel Export
from datetime import datetime
from pathlib import Path

# Requires openpyxl: conda install openpyxl
try:
    from openpyxl import Workbook
    from openpyxl.worksheet.table import Table, TableStyleInfo
    from openpyxl.utils.dataframe import dataframe_to_rows
    from openpyxl.utils import get_column_letter
except ImportError:
    print("openpyxl not installed. Run: conda install openpyxl")
    raise

# Define all queries
journey_queries = {
    "Event-Overview": """
        SELECT 
            name as Event_Type,
            COUNT(*) as Count,
            ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 1) as Percent
        FROM searches
        GROUP BY 1
        ORDER BY 2 DESC
    """,
    
    "Search-Funnel": """
        SELECT
            COUNT(DISTINCT CASE WHEN name IN ('SEARCH_TRIGGERED', 'SEARCH_STARTED') THEN session_key END) as Sessions_With_Search,
            COUNT(DISTINCT CASE WHEN name = 'SEARCH_RESULT_COUNT' THEN session_key END) as Sessions_With_Results,
            COUNT(DISTINCT CASE WHEN name IN ('SEARCH_TAB_CLICK', 'SEARCH_ALL_TAB_PAGE_CLICK', 'SEARCH_NEWS_TAB_PAGE_CLICK', 'SEARCH_GOTO_TAB_PAGE_CLICK') THEN session_key END) as Sessions_With_Click,
            ROUND(100.0 * COUNT(DISTINCT CASE WHEN name IN ('SEARCH_TAB_CLICK', 'SEARCH_ALL_TAB_PAGE_CLICK', 'SEARCH_NEWS_TAB_PAGE_CLICK', 'SEARCH_GOTO_TAB_PAGE_CLICK') THEN session_key END) 
                / NULLIF(COUNT(DISTINCT CASE WHEN name IN ('SEARCH_TRIGGERED', 'SEARCH_STARTED') THEN session_key END), 0), 1) as Click_Through_Rate_Pct
        FROM searches
    """,
    
    "Null-Results": """
        SELECT 
            COALESCE(CP_searchQuery, searchQuery, query) as Search_Term,
            COUNT(*) as Count,
            ROUND(AVG(CAST(CP_totalResultCount AS FLOAT)), 1) as Avg_Results,
            SUM(CASE WHEN CAST(CP_totalResultCount AS INTEGER) = 0 THEN 1 ELSE 0 END) as Null_Result_Count,
            ROUND(100.0 * SUM(CASE WHEN CAST(CP_totalResultCount AS INTEGER) = 0 THEN 1 ELSE 0 END) / COUNT(*), 1) as Null_Rate_Pct
        FROM searches
        WHERE name = 'SEARCH_RESULT_COUNT'
        GROUP BY 1
        HAVING COUNT(*) >= 3
        ORDER BY Null_Result_Count DESC
        LIMIT 100
    """,
    
    "Success-Rate": """
        WITH searches_with_query AS (
            SELECT 
                session_key,
                COALESCE(CP_searchQuery, searchQuery, query) as search_term,
                name
            FROM searches
            WHERE COALESCE(CP_searchQuery, searchQuery, query) IS NOT NULL
        ),
        search_events AS (
            SELECT DISTINCT session_key, search_term
            FROM searches_with_query 
            WHERE name IN ('SEARCH_TRIGGERED', 'SEARCH_STARTED', 'SEARCH_RESULT_COUNT')
        ),
        click_events AS (
            SELECT DISTINCT session_key
            FROM searches_with_query
            WHERE name IN ('SEARCH_TAB_CLICK', 'SEARCH_ALL_TAB_PAGE_CLICK', 'SEARCH_NEWS_TAB_PAGE_CLICK', 'SEARCH_GOTO_TAB_PAGE_CLICK')
        )
        SELECT 
            s.search_term as Search_Term,
            COUNT(*) as Search_Count,
            SUM(CASE WHEN c.session_key IS NOT NULL THEN 1 ELSE 0 END) as With_Click,
            ROUND(100.0 * SUM(CASE WHEN c.session_key IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*), 1) as Success_Rate_Pct
        FROM search_events s
        LEFT JOIN click_events c ON s.session_key = c.session_key
        GROUP BY 1
        HAVING COUNT(*) >= 5
        ORDER BY Search_Count DESC
        LIMIT 100
    """,
    
    "Abandoned-Searches": """
        WITH search_results AS (
            SELECT 
                session_key,
                COALESCE(CP_searchQuery, searchQuery, query) as search_term,
                CAST(CP_totalResultCount AS INTEGER) as total_results
            FROM searches
            WHERE name = 'SEARCH_RESULT_COUNT'
              AND CAST(CP_totalResultCount AS INTEGER) > 0
        ),
        click_events AS (
            SELECT DISTINCT session_key
            FROM searches
            WHERE name IN ('SEARCH_TAB_CLICK', 'SEARCH_ALL_TAB_PAGE_CLICK', 'SEARCH_NEWS_TAB_PAGE_CLICK', 'SEARCH_GOTO_TAB_PAGE_CLICK')
        )
        SELECT 
            sr.search_term as Search_Term,
            COUNT(*) as Search_Count,
            ROUND(AVG(sr.total_results), 0) as Avg_Results,
            SUM(CASE WHEN c.session_key IS NULL THEN 1 ELSE 0 END) as Without_Click,
            ROUND(100.0 * SUM(CASE WHEN c.session_key IS NULL THEN 1 ELSE 0 END) / COUNT(*), 1) as Abandon_Rate_Pct
        FROM search_results sr
        LEFT JOIN click_events c ON sr.session_key = c.session_key
        GROUP BY 1
        HAVING COUNT(*) >= 5 AND SUM(CASE WHEN c.session_key IS NULL THEN 1 ELSE 0 END) > 0
        ORDER BY Without_Click DESC
        LIMIT 100
    """,
    
    "Reformulations": """
        WITH session_searches AS (
            SELECT 
                session_key,
                COUNT(DISTINCT COALESCE(CP_searchQuery, searchQuery, query)) as unique_queries
            FROM searches
            WHERE name IN ('SEARCH_TRIGGERED', 'SEARCH_STARTED')
              AND COALESCE(CP_searchQuery, searchQuery, query) IS NOT NULL
            GROUP BY 1
            HAVING COUNT(DISTINCT COALESCE(CP_searchQuery, searchQuery, query)) > 1
        )
        SELECT 
            unique_queries as Different_Search_Count,
            COUNT(*) as Sessions,
            ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 1) as Percent
        FROM session_searches
        GROUP BY 1
        ORDER BY 1
    """,
    
    "Result-Categories": """
        SELECT
            'Total' as Category,
            COUNT(*) as Searches,
            ROUND(AVG(CAST(CP_totalResultCount AS FLOAT)), 1) as Avg_Count,
            SUM(CASE WHEN CAST(CP_totalResultCount AS INTEGER) = 0 THEN 1 ELSE 0 END) as Null_Results
        FROM searches WHERE name = 'SEARCH_RESULT_COUNT'
        
        UNION ALL
        
        SELECT 'People', COUNT(*), ROUND(AVG(CAST(CP_peopleResultCount AS FLOAT)), 1),
            SUM(CASE WHEN CAST(CP_peopleResultCount AS INTEGER) = 0 THEN 1 ELSE 0 END)
        FROM searches WHERE name = 'SEARCH_RESULT_COUNT' AND CP_peopleResultCount IS NOT NULL
        
        UNION ALL
        
        SELECT 'News', COUNT(*), ROUND(AVG(CAST(CP_newsResultCount AS FLOAT)), 1),
            SUM(CASE WHEN CAST(CP_newsResultCount AS INTEGER) = 0 THEN 1 ELSE 0 END)
        FROM searches WHERE name = 'SEARCH_RESULT_COUNT' AND CP_newsResultCount IS NOT NULL
        
        UNION ALL
        
        SELECT 'Intranet News', COUNT(*), ROUND(AVG(CAST(CP_intranetNewsResultCount AS FLOAT)), 1),
            SUM(CASE WHEN CAST(CP_intranetNewsResultCount AS INTEGER) = 0 THEN 1 ELSE 0 END)
        FROM searches WHERE name = 'SEARCH_RESULT_COUNT' AND CP_intranetNewsResultCount IS NOT NULL
        
        UNION ALL
        
        SELECT 'GoTo', COUNT(*), ROUND(AVG(CAST(CP_gotoResultCount AS FLOAT)), 1),
            SUM(CASE WHEN CAST(CP_gotoResultCount AS INTEGER) = 0 THEN 1 ELSE 0 END)
        FROM searches WHERE name = 'SEARCH_RESULT_COUNT' AND CP_gotoResultCount IS NOT NULL
    """,
    
    "Click-Distribution": """
        SELECT 
            name as Click_Type,
            COUNT(*) as Count,
            ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 1) as Percent
        FROM searches
        WHERE name IN ('SEARCH_TAB_CLICK', 'SEARCH_ALL_TAB_PAGE_CLICK', 'SEARCH_NEWS_TAB_PAGE_CLICK', 'SEARCH_GOTO_TAB_PAGE_CLICK')
        GROUP BY 1
        ORDER BY 2 DESC
    """,
    
    "Top-Search-Terms": """
        SELECT 
            COALESCE(CP_searchQuery, searchQuery, query) as Search_Term,
            COUNT(*) as Count,
            ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 2) as Percent
        FROM searches
        WHERE COALESCE(CP_searchQuery, searchQuery, query) IS NOT NULL
          AND name IN ('SEARCH_TRIGGERED', 'SEARCH_STARTED', 'SEARCH_RESULT_COUNT')
        GROUP BY 1
        ORDER BY 2 DESC
        LIMIT 100
    """
}

# Create Excel file
output_file = f'../output/search_journey_analysis_{datetime.now().strftime("%Y%m%d")}.xlsx'
output_path = Path(output_file)
output_path.parent.mkdir(parents=True, exist_ok=True)

wb = Workbook()
wb.remove(wb.active)  # Remove default sheet

table_style = TableStyleInfo(
    name="TableStyleMedium9",
    showFirstColumn=False,
    showLastColumn=False,
    showRowStripes=True,
    showColumnStripes=False
)

for sheet_name, sql in journey_queries.items():
    # Execute query
    df = query(sql)
    
    # Create sheet (max 31 characters for sheet name)
    ws = wb.create_sheet(title=sheet_name[:31])
    
    # Write DataFrame to sheet
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            ws.cell(row=r_idx, column=c_idx, value=value)
    
    # Auto-adjust column width
    for col_idx, column in enumerate(df.columns, 1):
        max_length = max(
            len(str(column)),
            df[column].astype(str).str.len().max() if len(df) > 0 else 0
        )
        ws.column_dimensions[get_column_letter(col_idx)].width = min(max_length + 2, 50)
    
    # Format as table
    if len(df) > 0:
        table_ref = f"A1:{get_column_letter(len(df.columns))}{len(df) + 1}"
        table = Table(displayName=sheet_name.replace("-", "_").replace(" ", "_"), ref=table_ref)
        table.tableStyleInfo = table_style
        ws.add_table(table)

# Save
wb.save(output_file)

print(f"Excel exported: {output_file}")
print(f"\nIncluded tabs:")
for name in journey_queries.keys():
    print(f"  • {name}")

---
## Performance Metrics

If you have a column with response time (e.g. `response_time`)

In [ ]:
# Response time statistics
# NOTE: Replace 'response_time' with your column

query("""
    SELECT
        COUNT(*) as total,
        ROUND(AVG(response_time), 2) as avg_ms,
        ROUND(MEDIAN(response_time), 2) as median_ms,
        ROUND(PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY response_time), 2) as p90_ms,
        ROUND(PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY response_time), 2) as p95_ms,
        ROUND(PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY response_time), 2) as p99_ms,
        ROUND(MAX(response_time), 2) as max_ms
    FROM searches
    WHERE response_time IS NOT NULL
""")

---
## Visualizations

If Matplotlib is installed (`conda install matplotlib`)

In [ ]:
if PLOTTING_AVAILABLE:
    # Entries per day
    daily = query("""
        SELECT
            DATE_TRUNC('day', timestamp)::DATE as date,
            COUNT(*) as count
        FROM searches
        GROUP BY 1
        ORDER BY 1
    """)
    
    fig, ax = plt.subplots(figsize=(14, 5))
    ax.plot(daily['date'], daily['count'], linewidth=2, color='steelblue')
    ax.fill_between(daily['date'], daily['count'], alpha=0.3, color='steelblue')
    ax.set_title('Entries per Day', fontsize=14, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("Matplotlib not installed. Run: conda install matplotlib")

In [ ]:
if PLOTTING_AVAILABLE:
    # Distribution by hour
    hourly = query("""
        SELECT
            EXTRACT(HOUR FROM timestamp)::INT as hour,
            COUNT(*) as count
        FROM searches
        GROUP BY 1
        ORDER BY 1
    """)
    
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.bar(hourly['hour'], hourly['count'], color='steelblue')
    ax.set_title('Distribution by Time of Day', fontsize=14, fontweight='bold')
    ax.set_xlabel('Hour')
    ax.set_ylabel('Count')
    ax.set_xticks(range(0, 24))
    plt.tight_layout()
    plt.show()

In [ ]:
if PLOTTING_AVAILABLE:
    # Top 10 values
    COLUMN = 'search_query'  # <-- Change this
    
    top = query(f"""
        SELECT {COLUMN} as value, COUNT(*) as count
        FROM searches
        WHERE {COLUMN} IS NOT NULL AND {COLUMN} != ''
        GROUP BY 1
        ORDER BY 2 DESC
        LIMIT 10
    """)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    y_pos = range(len(top))
    ax.barh(y_pos, top['count'], color='steelblue')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(top['value'])
    ax.invert_yaxis()
    ax.set_title(f'Top 10: {COLUMN}', fontsize=14, fontweight='bold')
    ax.set_xlabel('Count')
    plt.tight_layout()
    plt.show()

---
## Custom Queries

Write your own SQL queries here:

In [ ]:
# Your custom query here:
query("""
    SELECT *
    FROM searches
    LIMIT 10
""")

In [ ]:
# Another query:
query("""
    SELECT *
    FROM searches
    LIMIT 10
""")

---
## Export for Power BI

Parquet files are ideal for Power BI: smaller, faster, data types preserved.

In [ ]:
# Raw data export (all data)
from datetime import datetime
from pathlib import Path
import os

# Fixed filename for Power BI folder refresh
output_file = '../output/searches_raw.parquet'
output_path = Path(output_file)

# Delete old file if exists (avoids corrupt files)
if output_path.exists():
    output_path.unlink()

# Export
execute(f"COPY searches TO '{output_file}' (FORMAT PARQUET)")

# Verify file is valid
try:
    test_read = query(f"SELECT COUNT(*) as n FROM read_parquet('{output_file}')")
    row_count = test_read['n'][0]
    size_mb = os.path.getsize(output_file) / (1024 * 1024)
    
    print(f"Exported: {output_file}")
    print(f"Rows:     {row_count:,}")
    print(f"Size:     {size_mb:.1f} MB")
    print(f"Status:   Parquet file validated")
    print(f"\nIn Power BI: Get Data → Folder → {Path(output_file).parent}")
except Exception as e:
    print(f"ERROR: Parquet file is invalid: {e}")

In [ ]:
# Aggregated daily data (for long periods / trend dashboards)
output_file = '../output/searches_daily.parquet'
output_path = Path(output_file)

# Delete old file if exists
if output_path.exists():
    output_path.unlink()

execute(f"""
    COPY (
        SELECT 
            DATE_TRUNC('day', timestamp)::DATE as date,
            COUNT(*) as total_searches,
            COUNT(DISTINCT search_query) as unique_queries,
            SUM(CASE WHEN results_count = 0 THEN 1 ELSE 0 END) as null_results,
            ROUND(100.0 * SUM(CASE WHEN results_count = 0 THEN 1 ELSE 0 END) / COUNT(*), 2) as null_rate_pct
        FROM searches
        GROUP BY 1
        ORDER BY 1
    ) TO '{output_file}' (FORMAT PARQUET)
""")

# Verify
try:
    test_read = query(f"SELECT COUNT(*) as n FROM read_parquet('{output_file}')")
    days = test_read['n'][0]
    size_mb = os.path.getsize(output_file) / (1024 * 1024)
    
    print(f"Exported: {output_file}")
    print(f"Days:     {days}")
    print(f"Size:     {size_mb:.2f} MB")
    print(f"Status:   Parquet file validated")
except Exception as e:
    print(f"ERROR: Parquet file is invalid: {e}")

---
## Add New Data

If you want to import additional CSV files later:

In [ ]:
# Append new CSV to existing table
# NEW_CSV = '../data/new_data.csv'

# execute(f"""
#     INSERT INTO searches
#     SELECT * FROM read_csv('{NEW_CSV}', auto_detect=true)
# """)

# print(f"New data added. Total: {query('SELECT COUNT(*) FROM searches')['count_star()'][0]:,} rows")

---
## Cleanup

In [ ]:
# Close connection (at end of session)
con.close()
print("Connection closed")

---
## Notes

**My columns:**
- ...

**Findings:**
- ...

**Open questions:**
- ...